## CSC575 Project-

In [2]:
%load_ext watermark
%watermark -a "Ruiyu Hu" -d -v -m

Ruiyu Hu 2019-03-18 

CPython 3.6.7
IPython 7.0.1

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 58 Stepping 9, GenuineIntel
CPU cores  : 4
interpreter: 64bit


In [3]:
# read the pdf file
import PyPDF2 

# tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from textblob import TextBlob # pos_tag

# create inverse index

import math
import glob
import json
import operator
from collections import Counter,defaultdict


import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

**Create Tokenize**

In [50]:
#%%file tokenize.py
def get_file_names():
    files = []
    for file in glob.glob("../data/documents/*.pdf"):
        files.append(file)
    return files
#get_file_names()
###########################################################################
def convert(file):
    pdf_content = []

    pdf = PyPDF2.PdfFileReader(open(str(file),"rb"))
    # pdf may be more than one page
    num_pages = pdf.numPages
    count = 0
    text = ''
    while count < num_pages:
        pageObj = pdf.getPage(count)
        count +=1
        text += pageObj.extractText().replace('\n','')
    if text != '':
        text = text
            
    pdf_content.append(text)   
    return pdf_content

'''def convert(file):
    text = []
    with open(str(file), 'rb') as f:
        for line in f.readlines():
            text.append(line.decode("utf-8", "ignore").strip())
    return text'''

###########################################################################     
def tokenize(lst):            
    # create a list of token
    
    tokens = [None] * len(lst)
    for i in range(len(lst)):
        tokens[i] = clean_token(lst[i])
    tokens = [t for tok in tokens for t in tok] 
    return tokens

########################################################################### 
def clean_token(text):
    #porter = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    #snowball = nltk.SnowballStemmer('english')
        
    stopset = set(stopwords.words('english'))
    stopset.update(('year'))
    
    noun_lst = []

    for word,tag in (TextBlob(text).tags):
        if tag in ("NN", "NNS", "NNP", "NNPS","JJ"):
            word = word.lower()
            word = lemmatizer.lemmatize(word, pos = 'n')
            #word = porter.stem(word)
            #word = snowball.stem(word)
            if word not in stopset and word.isalpha() and len(word)>2:
                noun_lst.append(word)
    return noun_lst

In [64]:
#for _ in get_file_names()[:2]:
    #print(convert(_))

In [65]:
#for _ in get_file_names()[:2]:
    #print(tokenize(convert(_)))

In [ ]:
'''# set up env locally
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r'C:\Users\RayHu\ruiyu-gcp-4ac10836d3b1.json' 

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

def pos_clean(dir):
    lst = tokenize(dirt)
    
    client = language.LanguageServiceClient()
    # part-of-speech tags from list(enums.PartOfSpeech.Tag)
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')
    tags = ['NOUN']
    
    output = []
    for _ in lst:
        #output =[]
        #doc = ' '.join(_)
        document = language.types.Document(content = _, type=enums.Document.Type.PLAIN_TEXT)
        tokens = client.analyze_syntax(document).tokens
        for token in tokens:
            if pos_tag[token.part_of_speech.tag] in tags:
                output.append(token.text.content)
                
    c = Counter(output)
    query = [key for key, val in c.most_common(20)]
    
    return query'''

**Index Generator**

In [51]:
def make_idx(tokens, doc_name, idx, length):
    for term in set(tokens):
        idx[term].append([doc_name,tokens.count(term)])
        length[doc_name] = len(set(tokens))

def write(inverted_idx,len_idx):
    inv_idx_file = open("../data/indexes/inverted_idx.json","w")
    json.dump(inverted_idx,inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","w")
    json.dump(len_idx,len_idx_file)
    
def generate_idx():
    resume_files = get_file_names()
    inverted_index = defaultdict(list)
    length_index = defaultdict(list)
    
    for file in resume_files:
        make_idx(tokenize(convert(file)), file, inverted_index, length_index)
        
    write(inverted_index,length_index)
    print ("Indexes generated")
    

In [52]:
generate_idx()

Indexes generated


In [70]:
#inv_idx_file = open("../data/indexes/inverted_idx.json","r")
#inverted_idx = json.load(inv_idx_file)
#inverted_idx

In [71]:
#len_idx_file = open("../data/indexes/len_idx.json","r")
#len_idx = json.load(len_idx_file)
#len_idx

In [ ]:
'''inv_idx_file = open("../data/indexes/inverted_idx.json","r")
inv_indx = json.load(inv_idx_file)
# create dictionary
doc_freq ={}
for key in sorted(inv_indx.keys()):
    doc_freq[key] = sum(Counter(set(doc_id for doc_id, term in inv_indx[key])).values())

dictionary = pd.DataFrame.from_dict(doc_freq,orient='index', columns=['DocFreq'])
dictionary.head()'''

**BM25**

For query Q and document d, we have BM25 d of Q:

$  BM25_{score}(Q,d)=\frac{(k_2+1)qf_i}{k_2+qf_i}\times \frac{(k_1+1)\times f_i}{f_i+k_1(1-b+b\times l_d/avg\_l)}\times log_2\frac{(r_i+0.5)/(R-r_i+0.5)}{(n_i-r_i+0.5)/(N-n_i-R+r_i+0.5)} ​$


* $r_i$ is the # of relevant documents containing term i 
* $n_i$  is the # of docs containing term i
* N is the total # of docs in the collection
* R is the number of relevant documents for this query  (set to 0 if no relevancy info is known)
* $f_i$  is the frequency of term i in the doc under consideration
* $qf_i$ is the frequency of term i in the query
* $k_1$ determines how the tf component of the term weight changes as $f_i$
  increases. (if 0, then tf component is ignored.) 
* $k_2$ has a similar role for the query term weights. Typical values make the equation less sensitive to k2 than k1 because query term frequencies are much lower and less variable than doc term frequencies.
* K ($k_1(1-b+b\times l_d/avg\_l)$) is more complicated. Its role is basically to normalize the tf component by document length.
* b regulates the impact of length normalization. (0 means none; 1 is full normalization.) 

In [53]:
from math import log

'''
IR Book: 11.4.3
Fomula: 11.33
'''
'''
typical TREC value (Text Retrieval Conference (TREC).)
f1 = 1.2
k2 varies from 0 to 1000
b = 0.75
'''

k1 = 1.2
b = 0.75
k2 = 100
R = 0 # (set it to 0 since no relevancy info is known)



def BM25(doc_len, avg_doc_len, n_doc_w_term, n_total_doc, freq_term_doc, freq_term_query, rel_doc_w_term):
    
    n = n_doc_w_term
    N = n_total_doc
    f = freq_term_doc
    q = freq_term_query
    r = rel_doc_w_term
    
    p1 = ((k2 + 1) * q) / (k2 + q) #Relevance between term and query
    p2 = ((k1 + 1) * f) / (getK(doc_len, avg_doc_len) + f) #Relevance between term and document
    p3 = log((((r + 0.5)/(R-r+0.5)) / ((n - r + 0.5)/(N - n - R + r + 0.5)))+1) # Term Weight
    return p1 * p2 * p3

def getK(doc_len, avg_doc_len):
    return k1 * ((1 - b) + b * (float(doc_len) / float(avg_doc_len)))
    
#def sigmoid(x):
    #return 1 / (1 + math.exp(-x))

In [56]:
# get average document length
def get_avg_doc_len(len_idx):
    _length = 0
    for doc in len_idx:
        _length += len_idx[doc]
    return float(_length) / float(len(len_idx))

def search(query):
    inv_idx_file = open("../data/indexes/inverted_idx.json","r")
    inverted_idx = json.load(inv_idx_file)

    len_idx_file = open("../data/indexes/len_idx.json","r")
    len_idx = json.load(len_idx_file)

    scores = defaultdict(list)
    
    query_tokens = query.split()
    for token in query_tokens:
    #for token in query:
        #token = token.lower()
        for tok in clean_token(token):
            if tok in inverted_idx.keys():
                for entry in inverted_idx[tok]:
                    bm25_val = BM25(len_idx[entry[0]],get_avg_doc_len(len_idx),len(inverted_idx[tok]),len(len_idx),entry[1],1,0)
                    #scores[entry[0]] = round(10* sigmoid(bm25_val)-5,4)
                    scores[entry[0]] = round(bm25_val,4)
    result = sorted(scores.items(),key=operator.itemgetter(1),reverse=True)
    return result

In [74]:
#search('machine learning')[:5]

[('../data/documents\\ml002.pdf', 0.8568),
 ('../data/documents\\ml004.pdf', 0.8517),
 ('../data/documents\\ml003.pdf', 0.8479),
 ('../data/documents\\ml005.pdf', 0.8459),
 ('../data/documents\\ml001.pdf', 0.7893)]

**Cosine SImilarity**

In [9]:
import time
from gensim.models import KeyedVectors
t1 = time.time()
#download link: https://github.com/mmihaltz/word2vec-GoogleNews-vectors
path = 'C:/Users/RayHu/Downloads/google_w2v/GoogleNews-vectors-negative300.bin'
w2v_model = KeyedVectors.load_word2vec_format(path, binary=True)
print('-------------------------------------------')
print("Loading word2vec model cost %.3f seconds...\n" % (time.time() - t1))


-------------------------------------------
Loading word2vec model cost 86.375 seconds...



In [80]:
#%%file vectorize.py
import numpy as np
from numpy import linalg as LA

def vectorize(words):
    '''
    transform the doc and query into vectors
    '''
    word_vec = []
    for word in words:
        try:
            vec = w2v_model[word]
            word_vec.append(vec)
        except KeyError:
            # ignore the word if it is not in the w2v vocablary
            pass
    vector = np.mean(word_vec,axis = 0)
    return vector

def cos_sim(vector1, vector2):
    '''
    the fomula to calculte cosine cimilarity 
    '''
    sim = np.dot(vector1, vector2) / (LA.norm(vector1) * LA.norm(vector2))
    
    if np.isnan(np.sum(sim)):
        return 0
    
    return sim

def calc_sim(query):
    '''
    calculate similarity scores between documents and the query
    '''
    query = clean_token(query)
    file_list = get_file_names()
    documents = {}
    
    for i in range(len(file_list)):
        documents[file_list[i]] = tokenize(convert(file_list[i]))
        
    query_vec = vectorize(query)
    results = {}
    
    for name, doc in documents.items():
        doc_vec = vectorize(doc)
        sim_score = cos_sim(query_vec, doc_vec)
        #threshold = 0.5
        if sim_score > 0:
            results[name] = round(sim_score,4)
            sort_result = sorted(results.items(),key=operator.itemgetter(1),reverse=True)
    return sort_result

In [82]:
#calc_sim('machine learning')[:5]

**Jaccard Similarity**

In [78]:
def jaccard(query, documents):
    '''
    the fomula to calculate Jaccard
    '''
    s1 = set(query)
    s2 = set(documents)
    lst1 = s1.intersection(s2)
    lst2 = s1.union(s2)
    jaccard = 1.0 * len(lst1)/len(lst2)
    
    return jaccard

def calc_jac(query):
    query = set(clean_token(query))
    
    file_list = get_file_names()
    documents = {}
    
    for i in range(len(file_list)):
        documents[file_list[i]] = set(tokenize(convert(file_list[i])))
    
    results = {}
    
    for name, doc in documents.items():
        jac = jaccard(query, doc)
        results[name] = round(jac,4)
        sort_result = sorted(results.items(),key=operator.itemgetter(1),reverse=True)
    return sort_result

In [83]:
#calc_jac('machine learning')[:5]

**Output**

In [16]:
def matching(query):
    #print(query)
    print('-------------------------------------------------------')
    t1 = time.time()
    results_bm25 = search(query)[:3]
    print('bm25 result')
    for result in results_bm25:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t1))
    print('-------------------------------------------------------')
    t2 = time.time()
    results_cos = calc_sim(query)[:3]
    print('cosine result')
    for result in results_cos:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t2))   
    print('-------------------------------------------------------')
    t3 = time.time()
    results_jac = calc_jac(query)[:3]
    print('jacarrd result')
    for result in results_jac:
        print(result)
    print('The computing cost %.3f seconds\n'% (time.time() - t3))
    print('-------------------------------------------------------')

**Test for the long query**

In [26]:
twitter_ds_jd = 'Apply advanced statistical techniques to model user behavior, identify causal impact and attribution, build and benchmark metrics.Write complex data flows using SQL, Spark, Scalding, R and Python scripts.Use data visualization tools (e.g, Tableau, Zeppelin) to share ongoing insights.'
adobe_ml_jd = 'Hands on experience with Java, Python, and/or C++.Work on machine learning models & algorithms, web services, distributed systems, data mining, big data, Hadoop, deep learning, recommendations, and more by developing a Machine Platform at Adobe that would power Adobe Clouds.Apply data mining and machine learning to improve content understanding, computer vision, deep learning, language understanding and content ranking & recommendations.Maintain and optimize machine learning platform, identify new ideas to evolve it, develop new features and benchmark possible solutions.Build machine learning capabilities using technologies such as REST web services, micro-services, Caffe, Tensorflow, Spark, Elastic, AWS, Kafka, Deep Learning, Matlab, R, and more.'

In [84]:
queries=[twitter_ds_jd, adobe_ml_jd]    
for query in queries:
    print(query)
    matching(query)
    continue

Apply advanced statistical techniques to model user behavior, identify causal impact and attribution, build and benchmark metrics.Write complex data flows using SQL, Spark, Scalding, R and Python scripts.Use data visualization tools (e.g, Tableau, Zeppelin) to share ongoing insights.
-------------------------------------------------------
bm25 result
('../data/documents\\ds002.pdf', 1.361)
('../data/documents\\de005.pdf', 1.3028)
('../data/documents\\ds001.pdf', 1.2891)
The computing cost 0.180 seconds

-------------------------------------------------------
cosine result
('../data/documents\\ds002.pdf', 0.8088)
('../data/documents\\ds001.pdf', 0.8001)
('../data/documents\\ds005.pdf', 0.797)
The computing cost 3.793 seconds

-------------------------------------------------------
jacarrd result
('../data/documents\\ds003.pdf', 0.0663)
('../data/documents\\ds002.pdf', 0.0618)
('../data/documents\\de001.pdf', 0.0588)
The computing cost 4.904 seconds

-------------------------------------

In [87]:
'''queries=['machine learning','data science','data analysis','familiar wth python, sql and r']    
for query in queries:
    print(query)
    matching(query)
    continue'''

"queries=['machine learning','data science','data analysis','familiar wth python, sql and r']    \nfor query in queries:\n    print(query)\n    matching(query)\n    continue"